# Multiple regression (two predictors) with product of normals

- toc: true
- badges: true
- comments: true
- categories: [jupyter]

### About

I am trying to understand the underfitting in the VEB implementation of multiple regression with product of normals. Here, I will numerically check the underlying distributions for a special case with only two predictors.

In [2]:
#collapse-show

import numpy as np
import pandas as pd
from scipy import linalg as sc_linalg
from scipy import special as sc_special
import matplotlib.pyplot as plt

import mpl_stylesheet
import mpl_utils
from matplotlib import cm
from matplotlib import ticker as plticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
mpl_stylesheet.banskt_presentation(fontfamily = 'latex-clearsans', fontsize = 18, colors = 'banskt', dpi = 72)

In [ ]:
# collapse-hide

def variance_explained(y, ypred):
    ss_err = np.sum(np.square(y - ypred))
    ss_tot = np.sum(np.square(y - np.mean(y)))
    r2 = 1 - (ss_err / ss_tot)
    return r2

def prod_norm_prior_pdf(z, s1, s2):
    x = np.abs(z) / s1 / s2
    prob = sc_special.kn(0, x) / (np.pi * s1 * s2)
    return prob

def normal_logdensity_onesample(z, m, s2):
    logdensity = - 0.5 * np.log(2 * np.pi * s2) \
                 - 0.5 * (z - m) * (z - m) / s2
    return logdensity

def prod_norm_prior2_pdf(z, s1, s2):
    w = 3.5
    b = z / w
    p1 = np.exp(normal_logdensity_onesample(b, 0, s1 * s1))
    p2 = np.exp(normal_logdensity_onesample(w, 0, s2 * s2))
    return p1 * p2

def normal_logdensity(y, mean, sigma2):
    n = y.shape[0]
    logdensity = - 0.5 * n * np.log(2 * np.pi * sigma2) \
                 - 0.5 * np.sum(np.square(y - mean)) / sigma2
    return logdensity

def data_log_likelihood(X, y, b, sigma):
    ll = np.zeros_like(b)
    for i, _b in enumerate(b):
        ll[i] = normal_logdensity(y, X * _b, sigma * sigma)
    return ll